# Advanced Temporal Feature Engineering on Delta Lake Data

This notebook demonstrates how to apply the same temporal feature engineering we did on CSV data, but using PySpark on Delta Lake tables. This approach is more scalable for large datasets and integrates with your existing data lake architecture.

**Source Table:** `default.silver_flights_processed`
**Output Table:** `default.gold_flights_temporal`

## 1. Import Required Libraries

In [ ]:
from pyspark.sql.functions import (
    col, when, count, isnan, to_date, dayofweek, dayofmonth, month, year,
    weekofyear, date_format, datediff, expr, lit, array, array_contains,
    udf, collect_list, explode
)
from pyspark.sql.types import StringType, IntegerType, DateType
import holidays
from datetime import datetime, timedelta

print("✅ PySpark and temporal feature engineering libraries imported")

## 2. Load Data from Delta Lake

In [ ]:
# Load the Silver table from Delta Lake
print("Loading Silver table from Delta Lake...")
df_silver = spark.table("default.silver_flights_processed")

print(f"✅ Loaded {df_silver.count():,} flights from Delta Lake")
print(f"Columns: {len(df_silver.columns)}")

# Show schema
print("\nSchema:")
df_silver.printSchema()

# Show sample data
print("\nSample data:")
df_silver.show(5)

## 3. Apply Temporal Feature Engineering

In [ ]:
print("Applying temporal feature engineering...")

# First, ensure we have a proper date column
df_temporal = df_silver.withColumn("flight_date", to_date(col("FL_DATE")))

# Basic temporal features using PySpark SQL functions
df_temporal = df_temporal.withColumn("day_of_week", dayofweek(col("flight_date"))) \
                       .withColumn("day_of_week_name", date_format(col("flight_date"), "EEEE")) \
                       .withColumn("week_of_year", weekofyear(col("flight_date"))) \
                       .withColumn("is_weekend", when(col("day_of_week").isin([1, 7]), 1).otherwise(0))

print("✅ Basic temporal features added")

# Create holiday detection UDFs
def get_us_holidays_for_years(years):
    """Get all US holidays for given years"""
    all_holidays = {}
    for year in years:
        us_holidays = holidays.US(years=year)
        for date, name in us_holidays.items():
            if any(keyword in name.upper() for keyword in [
                'NEW YEAR', 'MARTIN LUTHER KING', 'WASHINGTON', 'MEMORIAL',
                'INDEPENDENCE', 'LABOR', 'COLUMBUS', 'VETERANS', 'THANKSGIVING', 'CHRISTMAS'
            ]):
                all_holidays[date.strftime('%Y-%m-%d')] = name
    return all_holidays

# Get unique years from the data
years_list = df_temporal.select(year(col("flight_date"))).distinct().rdd.flatMap(lambda x: x).collect()
print(f"Years in dataset: {sorted(years_list)}")

# Get holidays for all years
holidays_dict = get_us_holidays_for_years(years_list)
print(f"Total holidays identified: {len(holidays_dict)}")

# Create broadcast variable for holidays (efficient for distributed processing)
holidays_broadcast = spark.sparkContext.broadcast(holidays_dict)

# Create UDFs for holiday detection
def is_near_holiday_udf(date_str):
    """Check if date is near a holiday (within 3 days)"""
    if not date_str:
        return 0

    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        holidays_dict = holidays_broadcast.value

        for holiday_str in holidays_dict.keys():
            holiday_date = datetime.strptime(holiday_str, '%Y-%m-%d')
            if abs((date - holiday_date).days) <= 3:
                return 1
        return 0
    except:
        return 0

def nearest_holiday_udf(date_str):
    """Get name of nearest holiday"""
    if not date_str:
        return None

    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        holidays_dict = holidays_broadcast.value

        min_distance = float('inf')
        nearest_holiday = None

        for holiday_str, holiday_name in holidays_dict.items():
            holiday_date = datetime.strptime(holiday_str, '%Y-%m-%d')
            distance = abs((date - holiday_date).days)
            if distance <= 3 and distance < min_distance:
                min_distance = distance
                nearest_holiday = holiday_name

        return nearest_holiday
    except:
        return None

def is_holiday_period_udf(date_str):
    """Check if date is in holiday period (±7 pre, +3 post)"""
    if not date_str:
        return 0

    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        holidays_dict = holidays_broadcast.value

        for holiday_str in holidays_dict.keys():
            holiday_date = datetime.strptime(holiday_str, '%Y-%m-%d')
            days_diff = (date - holiday_date).days
            if -7 <= days_diff <= 3:
                return 1
        return 0
    except:
        return 0

def holiday_period_name_udf(date_str):
    """Get name of holiday in period"""
    if not date_str:
        return None

    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        holidays_dict = holidays_broadcast.value

        for holiday_str, holiday_name in holidays_dict.items():
            holiday_date = datetime.strptime(holiday_str, '%Y-%m-%d')
            days_diff = (date - holiday_date).days
            if -7 <= days_diff <= 3:
                return holiday_name
        return None
    except:
        return None

def days_to_holiday_udf(date_str):
    """Get days to nearest holiday in period"""
    if not date_str:
        return None

    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        holidays_dict = holidays_broadcast.value

        min_distance = float('inf')
        for holiday_str in holidays_dict.keys():
            holiday_date = datetime.strptime(holiday_str, '%Y-%m-%d')
            days_diff = (date - holiday_date).days
            if -7 <= days_diff <= 3 and abs(days_diff) < abs(min_distance):
                min_distance = days_diff

        return int(min_distance) if min_distance != float('inf') else None
    except:
        return None

# Register UDFs
is_near_holiday_spark_udf = udf(is_near_holiday_udf, IntegerType())
nearest_holiday_spark_udf = udf(nearest_holiday_udf, StringType())
is_holiday_period_spark_udf = udf(is_holiday_period_udf, IntegerType())
holiday_period_name_spark_udf = udf(holiday_period_name_udf, StringType())
days_to_holiday_spark_udf = udf(days_to_holiday_udf, IntegerType())

# Apply holiday UDFs
date_str_col = date_format(col("flight_date"), "yyyy-MM-dd")

df_temporal = df_temporal.withColumn("date_str", date_str_col) \
                       .withColumn("is_near_holiday", is_near_holiday_spark_udf(col("date_str"))) \
                       .withColumn("nearest_holiday", nearest_holiday_spark_udf(col("date_str"))) \
                       .withColumn("is_holiday_period", is_holiday_period_spark_udf(col("date_str"))) \
                       .withColumn("holiday_period_name", holiday_period_name_spark_udf(col("date_str"))) \
                       .withColumn("days_to_holiday", days_to_holiday_spark_udf(col("date_str"))) \
                       .drop("date_str")

print("✅ Holiday features added using PySpark UDFs")

# Show sample of new features
print("\nSample of temporal features:")
df_temporal.select(
    "flight_date", "day_of_week", "day_of_week_name", "week_of_year", "is_weekend",
    "is_near_holiday", "nearest_holiday", "is_holiday_period", "days_to_holiday"
).show(10)

# Show statistics
print("\nTemporal feature statistics:")
df_temporal.select(
    "is_weekend", "is_near_holiday", "is_holiday_period"
).summary().show()

## 4. Save Enhanced Data to Delta Lake

In [ ]:
# Define paths for the Gold table with temporal features
GOLD_TEMPORAL_PATH = "/Volumes/workspace/default/ds_capstone/gold/flights_temporal"
GOLD_TEMPORAL_TABLE_NAME = "default.gold_flights_temporal"
DATABASE_NAME = "default"

def path_exists(path):
    """Check if a path exists"""
    try:
        dbutils.fs.ls(path)
        return True
    except:
        return False

def create_directory_if_not_exists(path):
    """Create directory if it doesn't exist"""
    if not path_exists(path):
        dbutils.fs.mkdirs(path)
        print(f"✅ Created directory: {path}")
    else:
        print(f"ℹ️  Directory already exists: {path}")

# Check if Gold temporal path exists and clean if needed
print(f"\n📁 Checking Gold temporal path: {GOLD_TEMPORAL_PATH}")
if path_exists(GOLD_TEMPORAL_PATH):
    print(f"⚠️  Path already exists. Will overwrite existing table")
else:
    print(f"✅ Path is clear, ready to create new table")

# Create parent directory if needed
gold_temporal_parent = "/".join(GOLD_TEMPORAL_PATH.split("/")[:-1])
create_directory_if_not_exists(gold_temporal_parent)

# Write Gold temporal Delta table
print(f"\n💾 Writing Gold temporal Delta table...")
try:
    df_temporal.write.format("delta").mode("overwrite").save(GOLD_TEMPORAL_PATH)
    print(f"✅ Delta table written to: {GOLD_TEMPORAL_PATH}")
    print(f"✅ Records written: {df_temporal.count():,}")
except Exception as e:
    print(f"❌ ERROR: Could not write Delta table")
    print(f"   Error: {str(e)}")
    print(f"\n💡 Trying to clean and retry...")
    try:
        dbutils.fs.rm(GOLD_TEMPORAL_PATH, recurse=True)
        df_temporal.write.format("delta").mode("overwrite").save(GOLD_TEMPORAL_PATH)
        print(f"✅ Successfully wrote Delta table after cleanup")
    except Exception as e2:
        print(f"❌ Still failed: {str(e2)}")
        raise

# Register Delta table
print(f"\n📌 Registering Delta table as: {GOLD_TEMPORAL_TABLE_NAME}")
try:
    # Ensure database exists
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {DATABASE_NAME}")
    print(f"✅ Database '{DATABASE_NAME}' ready")

    # Drop table if it exists (to avoid conflicts)
    spark.sql(f"DROP TABLE IF EXISTS {GOLD_TEMPORAL_TABLE_NAME}")
    print(f"   Dropped existing table (if any)")

    # Create managed table
    df_for_table = spark.read.format("delta").load(GOLD_TEMPORAL_PATH)
    df_for_table.write.format("delta").mode("overwrite").saveAsTable(GOLD_TEMPORAL_TABLE_NAME)

    print(f"✅ Table registered successfully as '{GOLD_TEMPORAL_TABLE_NAME}'!")
except Exception as e:
    print(f"⚠️  Could not create table with saveAsTable, trying alternative method...")
    try:
        # Alternative: Create external table with explicit LOCATION
        spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {GOLD_TEMPORAL_TABLE_NAME}
            USING DELTA
            LOCATION '{GOLD_TEMPORAL_PATH}'
        """)
        print(f"✅ Table registered with LOCATION clause!")
    except Exception as e2:
        print(f"⚠️  Table registration failed: {str(e2)}")
        print(f"💡 You can still access the data directly using:")
        print(f"   spark.read.format('delta').load('{GOLD_TEMPORAL_PATH}')")

print("
🎉 **SUCCESS!** Delta Lake temporal feature engineering completed!"print(f"📊 **Final Dataset Summary:**")
print(f"   - Total flights: {df_temporal.count():,}")
print(f"   - Original features: {len(df_silver.columns)}")
print(f"   - New temporal features: 9")
print(f"   - Total features: {len(df_temporal.columns)}")
print(f"   - Table location: {GOLD_TEMPORAL_TABLE_NAME}")
print(f"   - Delta path: {GOLD_TEMPORAL_PATH}")

# Show final schema
print("
📋 **Final Schema:**"df_temporal.printSchema()